In [1]:
from huggingface_hub import notebook_login
import wandb

notebook_login()
wandb.login()

wandb: Currently logged in as: kimitoinf. Use `wandb login --relogin` to force relogin


True

In [2]:
import os
import random
import numpy as np
import torch

def set_seed(seed_value = 42):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed()

In [3]:
from datasets import load_dataset

dataset_path = './disease_data'
for label in os.listdir(dataset_path):
    print(label + ': ' + str(len(os.listdir(os.path.join(dataset_path, label)))))
dataset = load_dataset(dataset_path, split = 'train').shuffle(seed = 42).train_test_split(test_size = 0.3)
split = dataset['test'].train_test_split(test_size = 0.5)
dataset['validation'] = split['train']
dataset['test'] = split['test']
print(dataset)

none: 10800
corneal_ulcer: 5386
ulcerative_keratitis: 7727
epiphora: 10797
cataract: 7758
entropion: 8667


Resolving data files:   0%|          | 0/51135 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 35794
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 7671
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 7670
    })
})


In [4]:
labels = dataset['train'].features['label'].names
label2id, id2label = dict(), dict()
for loop, label in enumerate(labels):
    label2id[label] = str(loop)
    id2label[str(loop)] = label

In [5]:
from transformers import AutoImageProcessor, DefaultDataCollator
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor, RandomVerticalFlip, RandomHorizontalFlip

checkpoint = "google/vit-base-patch16-224-in21k"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

normalize = Normalize(mean = image_processor.image_mean, std = image_processor.image_std)
size = (
    image_processor.size["shortest_edge"]
    if "shortest_edge" in image_processor.size
    else (image_processor.size["height"], image_processor.size["width"])
)
_transforms = Compose([RandomResizedCrop(size), RandomVerticalFlip(), RandomHorizontalFlip(), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]
    return examples

dataset = dataset.with_transform(transforms)
data_collator = DefaultDataCollator()

print(dataset['train'][0])

/shared_hdd/kimitoinf/miniconda3/envs/kimit/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'label': 3, 'pixel_values': tensor([[[-0.1451, -0.0980, -0.0588,  ...,  0.2235,  0.0667, -0.0510],
         [-0.0588, -0.0745, -0.0588,  ...,  0.1451, -0.0588, -0.1373],
         [ 0.0039, -0.0510, -0.1059,  ..., -0.0588, -0.1059, -0.0588],
         ...,
         [-0.0275,  0.0196,  0.1137,  ..., -0.0039, -0.0353,  0.0902],
         [ 0.1765,  0.2000,  0.2235,  ...,  0.1686,  0.1451,  0.1922],
         [ 0.1451,  0.1765,  0.0667,  ...,  0.3098,  0.2941,  0.2549]],

        [[-0.4431, -0.3961, -0.3647,  ...,  0.0353, -0.1294, -0.2784],
         [-0.3569, -0.3725, -0.3647,  ..., -0.0588, -0.2706, -0.3490],
         [-0.2941, -0.3490, -0.4039,  ..., -0.2706, -0.3176, -0.2549],
         ...,
         [-0.3647, -0.3176, -0.2235,  ..., -0.2863, -0.3020, -0.1686],
         [-0.1608, -0.1373, -0.1137,  ..., -0.1137, -0.1294, -0.0588],
         [-0.1922, -0.1608, -0.2706,  ...,  0.0353,  0.0196,  0.0039]],

        [[-0.6078, -0.5451, -0.4980,  ..., -0.1608, -0.3255, -0.4824],
         [-0.513

In [6]:
import numpy as np
import torch
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis = -1)
    accuracy = evaluate.load('accuracy').compute(predictions = predictions, references = labels)
    f1_precision_recall = evaluate.combine(['f1', 'precision', 'recall']).compute(predictions = predictions, references = labels, average = 'weighted')
    metrics = dict()
    for loop in [accuracy, f1_precision_recall]:
        metrics.update(loop)
    probs = np.array(torch.nn.functional.softmax(torch.tensor(logits), dim = -1).tolist())
    classes = list(id2label.values())
    true_labels = np.array(labels)
    wandb.log({
        'roc': wandb.plot.roc_curve(y_true = true_labels, y_probas = probs, labels = classes),
        'pr': wandb.plot.pr_curve(y_true = true_labels, y_probas = probs, labels = classes)
    })
    # wandb.sklearn.plot_confusion_matrix(y_true = true_labels, y_pred = np.array(predictions), labels = classes) # graph is overlapped.
    return metrics

In [7]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer
from PIL import ImageFile
import os
import torch

ImageFile.LOAD_TRUNCATED_IMAGES = True
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
os.environ['WANDB_PROJECT'] = 'dedc'
torch.cuda.empty_cache()

model = AutoModelForImageClassification.from_pretrained(
    checkpoint,
    num_labels = len(labels),
    id2label = id2label,
    label2id = label2id,
)

def train():
    training_args = TrainingArguments(
        output_dir = "./dedc",
        remove_unused_columns = False,
        eval_strategy = "epoch",
        save_strategy = "epoch",
        learning_rate = 5e-5,
        per_device_train_batch_size = 16,
        gradient_accumulation_steps = 4,
        per_device_eval_batch_size = 16,
        num_train_epochs = 25,
        warmup_ratio = 0.1,
        logging_steps = 10,
        load_best_model_at_end = True,
        metric_for_best_model = "accuracy",
        push_to_hub = True,
        run_name = 'run',
        report_to = 'wandb'
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = dataset['train'],
        eval_dataset = dataset['validation'],
        tokenizer = image_processor,
        compute_metrics = compute_metrics,
    )

    trainer.train()
    wandb.finish()

train()
model_path = './model'
image_processor.save_pretrained(model_path)
model.save_pretrained(model_path)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.


wandb: Tracking run with wandb version 0.17.0


wandb: Run data is saved locally in /shared_hdd/kimitoinf/dedc/wandb/run-20240522_120208-no93rkly
wandb: Run `wandb offline` to turn off syncing.


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,0.735300,0.734238,0.729465,0.726390,0.753608,0.729465
2,0.513400,0.523651,0.792438,0.789042,0.789947,0.792438
4,0.382500,0.458118,0.812256,0.806163,0.808838,0.812256
6,0.405800,0.426109,0.821382,0.820830,0.823951,0.821382
8,0.337000,0.389228,0.830117,0.830433,0.832359,0.830117
10,0.268300,0.389591,0.832595,0.827919,0.829108,0.832595
12,0.321800,0.375604,0.843677,0.844018,0.844517,0.843677
14,0.291900,0.369633,0.844198,0.842871,0.843848,0.844198
16,0.241600,0.354240,0.848892,0.848115,0.848017,0.848892
18,0.232200,0.356578,0.844850,0.844648,0.844690,0.844850


wandb:                                                                                


wandb: 
wandb: Run history:
wandb:           eval/accuracy ▁▃▅▅▆▇▆▇▇▇▇██▇████████▇██
wandb:                 eval/f1 ▁▃▅▅▆▇▆▆▇▇▇██▇████████▇██
wandb:               eval/loss █▆▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:          eval/precision ▁▂▄▅▅▆▆▆▇▇▇██▇████████▇██
wandb:             eval/recall ▁▃▅▅▆▇▆▇▇▇▇██▇████████▇██
wandb:            eval/runtime ▆█▇▇▄▅▄▅▅▄▆▅▆▆▆▄▄▁▁▁▁▁▁▁▁
wandb: eval/samples_per_second ▁▁▁▁▂▂▂▂▂▂▁▂▁▁▁▂▂█████▇▇▇
wandb:   eval/steps_per_second ▁▁▁▁▂▂▂▂▂▂▁▂▁▁▁▂▂█████▇▇▇
wandb:             train/epoch ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:       train/global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:         train/grad_norm ▂▃▄▃▃▂▄▂▄▃▃▄▃▅▃▄▅▄▃▄▂▂▃▅▃▃▂▃▂▃▃▄▂▃▃▁▁█▁▅
wandb:     train/learning_rate ▂▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁
wandb:              train/loss █▄▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:            eval/accuracy 0.84302
wandb:                  eval/f1 0.84268
wandb:                eval/loss 0.35914
wandb:  

wandb: 🚀 View run run at: https://wandb.ai/kimitoinf/dedc/runs/no93rkly
wandb: ⭐️ View project at: https://wandb.ai/kimitoinf/dedc
wandb: Synced 5 W&B file(s), 50 media file(s), 53 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240522_120208-no93rkly/logs


In [8]:
from transformers import pipeline
from torchvision.transforms.functional import to_pil_image
from tqdm import notebook
import wandb

wandb.init(project = 'dedc', name = 'test')

model_path = './model'
classifier = pipeline("image-classification", model = model_path)

true_labels = []
pred_labels = []
for loop in notebook.tqdm(dataset['test']):
    prediction = classifier(to_pil_image(loop['pixel_values']))
    true_labels.append(loop['label'])
    pred_labels.append(int(label2id[prediction[0]['label']]))

accuracy = evaluate.load('accuracy').compute(predictions = pred_labels, references = true_labels)
f1_precision_recall = evaluate.combine(['f1', 'precision', 'recall']).compute(predictions = pred_labels, references = true_labels, average = 'weighted')
metrics = dict()
for loop in [accuracy, f1_precision_recall]:
    metrics.update(loop)
wandb.log(metrics)
wandb.sklearn.plot_confusion_matrix(y_true = np.array(true_labels), y_pred = np.array(pred_labels), labels = list(id2label.values()))
wandb.finish()

wandb: WARNING Changes to your `wandb` environment variables will be ignored because your `wandb` session has already started. For more information on how to modify your settings with `wandb.init()` arguments, please refer to https://wandb.me/wandb-init.


wandb: Tracking run with wandb version 0.17.0


wandb: Run data is saved locally in /shared_hdd/kimitoinf/dedc/wandb/run-20240523_123519-fxaffk8u
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run test


wandb: ⭐️ View project at https://wandb.ai/kimitoinf/dedc


...

  0%|          | 0/7671 [00:00<?, ?it/s]

wandb:                                                                                


wandb: 
wandb: Run history:
wandb:  accuracy ▁
wandb:        f1 ▁
wandb: precision ▁
wandb:    recall ▁
wandb: 
wandb: Run summary:
wandb:  accuracy 0.59953
wandb:        f1 0.57884
wandb: precision 0.6325
wandb:    recall 0.59953
wandb: 


wandb: 🚀 View run test at: https://wandb.ai/kimitoinf/dedc/runs/fxaffk8u
wandb: ⭐️ View project at: https://wandb.ai/kimitoinf/dedc
wandb: Synced 5 W&B file(s), 1 media file(s), 1 artifact file(s) and 0 other file(s)


wandb: Find logs at: ./wandb/run-20240523_123519-fxaffk8u/logs
